This code generates keys.

In [2]:
# generate_keys.py
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa

# Generate private key
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)

# Save private key
with open("mykeys/private_key.pem", "wb") as f:
    f.write(
        private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption()
        )
    )

# Save public key
public_key = private_key.public_key()
with open("mykeys/public_key.pem", "wb") as f:
    f.write(
        public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )
    )

print("✅ Keys saved: private_key.pem, public_key.pem")

✅ Keys saved: private_key.pem, public_key.pem


I worked with my classmate Jeremi to try sending messages over LAN. Here I send a message to Jeremi. Jeremi needed to send me his local IP address so that I could send my messages to the right place.

In [6]:
# sender.py
import socket
import os
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import pickle

# Load recipient's public key
with open("friendkeys/public_key.pem", "rb") as f:
    public_key = serialization.load_pem_public_key(f.read())

# Message to send
message = b"Hello Jeremi, did you get my message?"

# 1. Generate random AES key and IV
aes_key = os.urandom(32) # AES-256
iv = os.urandom(16)

# 2. Encrypt message with AES (CFB mode)
cipher = Cipher(algorithms.AES(aes_key), modes.CFB(iv))
encryptor = cipher.encryptor()
encrypted_message = encryptor.update(message) + encryptor.finalize()

# 3. Encrypt AES key with RSA (recipient's public key)
encrypted_key = public_key.encrypt(
    aes_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# 4. Package: (encrypted_key, iv, encrypted_message)
payload = pickle.dumps((encrypted_key, iv, encrypted_message))

# 5. Send via socket
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect(("172.25.128.75", 65432))
    s.sendall(payload)
    print("✅ Encrypted message sent!")

    

✅ Encrypted message sent!


Here Jeremi sends me a message back. The reciever (server) listens to all IPs on this network, so it can recieve a message from any sender.

In [8]:
# receiver.py (server)
import socket
import pickle
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

# Load private key
with open("mykeys/private_key.pem", "rb") as f:
    private_key = serialization.load_pem_private_key(f.read(), password=None)

# Start server
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server:
    server.bind(("0.0.0.0", 65432))
    server.listen()
    print("Waiting for connection...")
    connection, address = server.accept()
    with connection:
        print(f"Connected by {address}")
        data = b""
        while True:
            chunk = connection.recv(4096)
            if not chunk:
                break
            data += chunk

# Unpack payload
encrypted_key, iv, encrypted_message = pickle.loads(data)

# 1. Decrypt AES key with RSA private key
aes_key = private_key.decrypt(
    encrypted_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# 2. Decrypt message with AES
cipher = Cipher(algorithms.AES(aes_key), modes.CFB(iv))
decryptor = cipher.decryptor()
message = decryptor.update(encrypted_message) + decryptor.finalize()

print("Decrypted message:", message.decode())

Waiting for connection...
Connected by ('172.25.128.75', 5982)
Decrypted message: Hi Maks! Yes i did!
